# Examples for functions from utils.general module.

Some functions have optional parameter mode (=MODE_NP by default). If mode is MODE_NP (=0) then numpy format is used for all calculations, if mode is MODE_TT (=1) then tensor train (TT) format is used, if mode is MODE_SP (=2) then scipy sparse format is used.

In [1]:
import numpy as np
from scipy.sparse import diags as sp_diag

import tt

from qttpdesolver.utils.general import MODE_NP, MODE_TT, MODE_SP
from qttpdesolver.utils.general import is_mode_np, is_mode_tt, is_mode_sp
from qttpdesolver.utils.general import define_mode, define_mode_list
from qttpdesolver.utils.general import rel_err, norm, ttround, vdiag, msum, vsum, mprod, vprod, mvec, vinv

## Vectors and matrices for examples

The vectors and matrices are constructed here in numpy format and then are transformed to [quantized] tensor train format (the last ones are named according to mask *_tt).

In [2]:
d = 5
n = 2**d
tau = 1.E-8
x = np.arange(n)+1.
y =-2.*x
z = x*y

x_tt = tt.tensor(x.reshape([2]*d, order='F'), tau)
y_tt = tt.tensor(y.reshape([2]*d, order='F'), tau)
z_tt = tt.tensor(z.reshape([2]*d, order='F'), tau)

A_tt = tt.Toeplitz(x_tt, d, kind='U')
A = A_tt.full()

B_tt = tt.Toeplitz(y_tt, d, kind='L')
B = B_tt.full()

C_tt = tt.Toeplitz(z_tt, d, kind='L')
C = C_tt.full()

I = np.eye(n)
I_sp = sp_diag([np.ones(n)], [0], format='csr')

## Examples

In [3]:
help(is_mode_np)
help(define_mode)
help(define_mode_list)

print is_mode_np(1),
print is_mode_np(x),
print is_mode_np(A),
print is_mode_np(x_tt)

print is_mode_tt(x),
print is_mode_tt(x_tt),
print is_mode_tt(A_tt)

print is_mode_sp(I_sp)

print define_mode(x),
print define_mode(A),
print define_mode(x_tt),
print define_mode(A_tt),
print define_mode(I_sp),
print define_mode('abc'),
print define_mode([1, 2, 3])

print define_mode_list([[None, z, x]]),
print define_mode_list([[None, None, None], [None, z_tt, z]]),
print define_mode_list([[None, I_sp, None], [None, z, z_tt]])

Help on function is_mode_np in module qttpdesolver.utils.general:

is_mode_np(x)
    Return True if x is in MODE_NP and False otherwise.

Help on function define_mode in module qttpdesolver.utils.general:

define_mode(x)
    Return MODE_NP / MODE_TT / MODE_SP or None according to the type of x.

Help on function define_mode_list in module qttpdesolver.utils.general:

define_mode_list(lst, num_as_none=False)
    Return MODE_NP / MODE_TT / MODE_SP or None according to the type of
    the first non-None list element.
    If num_as_none==True, then elements of int/float will be passed (as None).

True True True False
False True True
True
0 0 1 1 2 None None
0 1 2


In [4]:
help(norm)
print norm(np.ones(100)),
print norm(np.ones((100, 100))*2.),
print norm(x),
print norm(x_tt),
print norm(I_sp),
print norm(None)

Help on function norm in module qttpdesolver.utils.general:

norm(x)
    Return ||x|| if x != None or return None otherwise.

10.0 200.0 106.957935657 106.957935657 5.65685424949 None


In [5]:
help(rel_err)
print rel_err(x, 2.*x),
print rel_err(x_tt, 2.*x_tt),
print rel_err(2.*x, x),
print rel_err(2.*x_tt, x_tt),
print rel_err(A_tt, 2.*A_tt),
print rel_err(2.*A_tt, A_tt),
print rel_err(2.*I_sp, I_sp),
print rel_err(x, y),
print rel_err(x, None),
print rel_err(None, None)

Help on function rel_err in module qttpdesolver.utils.general:

rel_err(x1, x2)
    Return ||x1-x2||/||x1|| if x1 and x2 != None or return None otherwise.

1.0 1.0 0.5 0.5 1.0 0.5 0.5 3.0 None None


In [6]:
help(ttround)
print ttround(x_tt, tau=1.E-8).r
print ttround(x_tt, tau=1.E-1).r

Help on function ttround in module qttpdesolver.utils.general:

ttround(x=None, tau=None)
    Round tt-vector or matrix to accuracy tau,
    or return x if it is None or mode is MODE_NP or MODE_SP or if tau is None.

[1 2 2 2 2 1]
[1 1 1 2 2 1]


In [7]:
help(vdiag)
print vdiag(x, tau)
print vdiag(x_tt, tau).full()
print vdiag(B, tau)
print vdiag(B_tt, tau).full().flatten('F')
print vdiag(x, to_sp=True)
print vdiag(I_sp)

Help on function vdiag in module qttpdesolver.utils.general:

vdiag(x, tau=None, to_sp=False)
    Construct a diagonal matrix from a vector (if x is a vector) or
    construct matrix diagonal (if x is a matrix).
    If x is a vector in MODE_NP and result is needed in MODE_SP, then
    to_sp should be set to True.

[[  1.   0.   0. ...,   0.   0.   0.]
 [  0.   2.   0. ...,   0.   0.   0.]
 [  0.   0.   3. ...,   0.   0.   0.]
 ..., 
 [  0.   0.   0. ...,  30.   0.   0.]
 [  0.   0.   0. ...,   0.  31.   0.]
 [  0.   0.   0. ...,   0.   0.  32.]]
[[  1.   0.   0. ...,   0.   0.   0.]
 [  0.   2.   0. ...,   0.   0.   0.]
 [  0.   0.   3. ...,   0.   0.   0.]
 ..., 
 [  0.   0.   0. ...,  30.   0.   0.]
 [  0.   0.   0. ...,   0.  31.   0.]
 [  0.   0.   0. ...,   0.   0.  32.]]
[-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
[-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2.

In [8]:
help(msum)
print msum([x, y, z], tau)
print msum([x_tt, y_tt, z_tt], tau).full().flatten('F')
print msum([A, B, C], tau)
print msum([A_tt, B_tt, C_tt], tau).full()
print vdiag(msum([I_sp]*4, tau))

Help on function msum in module qttpdesolver.utils.general:

msum(lst, tau=None)
    Calculate a sum of a list of vectors or matrices.

[   -3.   -10.   -21.   -36.   -55.   -78.  -105.  -136.  -171.  -210.
  -253.  -300.  -351.  -406.  -465.  -528.  -595.  -666.  -741.  -820.
  -903.  -990. -1081. -1176. -1275. -1378. -1485. -1596. -1711. -1830.
 -1953. -2080.]
[   -3.   -10.   -21.   -36.   -55.   -78.  -105.  -136.  -171.  -210.
  -253.  -300.  -351.  -406.  -465.  -528.  -595.  -666.  -741.  -820.
  -903.  -990. -1081. -1176. -1275. -1378. -1485. -1596. -1711. -1830.
 -1953. -2080.]
[[ -4.00000000e+00  -1.20000000e+01  -2.40000000e+01 ...,  -1.86000000e+03
   -1.98400000e+03  -2.11200000e+03]
 [  3.20000000e+01  -4.00000000e+00  -1.20000000e+01 ...,  -1.74000000e+03
   -1.86000000e+03  -1.98400000e+03]
 [  3.10000000e+01   3.20000000e+01  -4.00000000e+00 ...,  -1.62400000e+03
   -1.74000000e+03  -1.86000000e+03]
 ..., 
 [  4.00000000e+00   5.00000000e+00   6.00000000e+00 ...,  -4.0

In [9]:
help(vsum)
print vsum(x, tau)
print vsum(x_tt, tau)

Help on function vsum in module qttpdesolver.utils.general:

vsum(x, tau=None)
    Calculate a sum of vector's elements.

528.0
528.0


In [10]:
help(mprod)
print mprod([A, B, C])
print mprod([A_tt, B_tt, C_tt], tau).full()
print vdiag(mprod([I_sp]*3+[-1.*I_sp], tau))

Help on function mprod in module qttpdesolver.utils.general:

mprod(lst, tau=None)
    Calculate matrix-dot product for a list of matrices m.

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.28000000e+02   7.68000000e+02   2.56000000e+03 ...,   9.84064000e+06
    1.11738880e+07   1.26382080e+07]
 [  1.24000000e+02   8.72000000e+02   3.24800000e+03 ...,   1.81635200e+07
    2.06653440e+07   2.34171520e+07]
 ..., 
 [  1.60000000e+01   1.16000000e+02   4.64000000e+02 ...,   1.80592280e+07
    2.15175360e+07   2.55061960e+07]
 [  1.20000000e+01   8.80000000e+01   3.56000000e+02 ...,   1.59969920e+07
    1.91067800e+07   2.27023680e+07]
 [  8.00000000e+00   6.00000000e+01   2.48000000e+02 ...,   1.39346240e+07
    1.66953600e+07   1.98966680e+07]]
[[ -1.39396434e-07   1.62286597e-07  -6.98928488e-08 ...,   7.91535740e-07
    3.94844392e-07   7.00880264e-07]
 [  1.28000000e+02   7.68000000e+02   2.56000000e+03 ...,   9.840

In [11]:
help(vprod)
print vprod([x, y, z], tau)
print vprod([x_tt, y_tt, z_tt], tau).full().flatten('F')

Help on function vprod in module qttpdesolver.utils.general:

vprod(lst, tau=None)
    Calculate elemenwise product for a list of vectors v.

[  4.00000000e+00   6.40000000e+01   3.24000000e+02   1.02400000e+03
   2.50000000e+03   5.18400000e+03   9.60400000e+03   1.63840000e+04
   2.62440000e+04   4.00000000e+04   5.85640000e+04   8.29440000e+04
   1.14244000e+05   1.53664000e+05   2.02500000e+05   2.62144000e+05
   3.34084000e+05   4.19904000e+05   5.21284000e+05   6.40000000e+05
   7.77924000e+05   9.37024000e+05   1.11936400e+06   1.32710400e+06
   1.56250000e+06   1.82790400e+06   2.12576400e+06   2.45862400e+06
   2.82912400e+06   3.24000000e+06   3.69408400e+06   4.19430400e+06]
[  4.00000000e+00   6.40000000e+01   3.24000000e+02   1.02400000e+03
   2.50000000e+03   5.18400000e+03   9.60400000e+03   1.63840000e+04
   2.62440000e+04   4.00000000e+04   5.85640000e+04   8.29440000e+04
   1.14244000e+05   1.53664000e+05   2.02500000e+05   2.62144000e+05
   3.34084000e+05   4.1990400

In [12]:
help(mvec)
print mvec(A, x, tau)
print mvec(A_tt, x_tt, tau).full().flatten('F')
print mvec(I_sp, x)

Help on function mvec in module qttpdesolver.utils.general:

mvec(A, x, tau=None)
    Calculate a matrix-vector product for matrix A and vector x.

[     0.     32.     95.    188.    310.    460.    637.    840.   1068.
   1320.   1595.   1892.   2210.   2548.   2905.   3280.   3672.   4080.
   4503.   4940.   5390.   5852.   6325.   6808.   7300.   7800.   8307.
   8820.   9338.   9860.  10385.  10912.]
[ -7.84439180e-12   3.20000000e+01   9.50000000e+01   1.88000000e+02
   3.10000000e+02   4.60000000e+02   6.37000000e+02   8.40000000e+02
   1.06800000e+03   1.32000000e+03   1.59500000e+03   1.89200000e+03
   2.21000000e+03   2.54800000e+03   2.90500000e+03   3.28000000e+03
   3.67200000e+03   4.08000000e+03   4.50300000e+03   4.94000000e+03
   5.39000000e+03   5.85200000e+03   6.32500000e+03   6.80800000e+03
   7.30000000e+03   7.80000000e+03   8.30700000e+03   8.82000000e+03
   9.33800000e+03   9.86000000e+03   1.03850000e+04   1.09120000e+04]
[  1.   2.   3.   4.   5.   6.   7.   

In [13]:
help(vinv)
eps = tau
print vinv(x, eps, tau)
print vinv(-x, abs_val=True)
print vinv(x_tt, eps, tau, verb=True).full().flatten('F')
print vinv(x_tt, eps, tau, x0=x_tt, verb=True, name='My vector').full().flatten('F')

Help on function vinv in module qttpdesolver.utils.general:

vinv(x, eps=1e-08, tau=None, x0=None, abs_val=False, name='Unknown vector', verb=False)
    For a given vector x calculate 1/x or 1/|x|.

[ 1.          0.5         0.33333333  0.25        0.2         0.16666667
  0.14285714  0.125       0.11111111  0.1         0.09090909  0.08333333
  0.07692308  0.07142857  0.06666667  0.0625      0.05882353  0.05555556
  0.05263158  0.05        0.04761905  0.04545455  0.04347826  0.04166667
  0.04        0.03846154  0.03703704  0.03571429  0.03448276  0.03333333
  0.03225806  0.03125   ]
[ 1.          0.5         0.33333333  0.25        0.2         0.16666667
  0.14285714  0.125       0.11111111  0.1         0.09090909  0.08333333
  0.07692308  0.07142857  0.06666667  0.0625      0.05882353  0.05555556
  0.05263158  0.05        0.04761905  0.04545455  0.04347826  0.04166667
  0.04        0.03846154  0.03703704  0.03571429  0.03448276  0.03333333
  0.03225806  0.03125   ]
  Construction of i